# Natural Language Processing

## Tugas 3: Information Retrieval

### Mekanisme

Anda hanya diwajibkan untuk mengumpulkan file ini saja ke uploader yang disediakan di https://elearning.uai.ac.id/. Ganti nama file ini saat pengumpulan menjadi **tugas3_NIM.ipynb**.

**Keterlambatan**: Pengumpulan tugas yang melebihi tenggat yang telah ditentukan tidak akan diterima. Keterlambatan akan berakibat pada nilai nol untuk tugas ini.

**Kolaborasi**: Anda diperbolehkan untuk berdiskusi dengan teman Anda, tetapi dilarang keras menyalin kode maupun tulisan dari teman Anda.

### Petunjuk

Pastikan jawaban Anda singkat, padat, dan jelas. Mayoritas pertanyaan yang diberikan dapat dijawab dalam 3-4 kalimat saja.

### Catatan ⚠️

1. Anda mungkin membutuhkan akses GPU untuk mengerjakan tugas ini. Untuk memudahkan, Anda dapat menggunakan akses GPU gratis dari [Google Colab](https://colab.research.google.com/).
2. Untuk bagian 3 dari tugas ini, Anda memerlukan akses ke model [Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Jalani cell ini untuk menginstalasi dependencies untuk memuat data dan menjalankan model
!pip install huggingface_hub fastparquet pyarrow rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00


## 1. Spare Matrix Retrieval (15 poin)

*Acknowledgement:* Data yang digunakan dalam tugas ini adalah [MS MARCO](https://huggingface.co/datasets/microsoft/ms_marco), sebuah koleksi dataset yang digunakan untuk mengembangkan model deep learning dalam kasus pencarian. Kumpulan data ini adalah jawaban pertanyaan berdasarkan 100.000 pertanyaan Bing asli dan jawaban yang dibuat manusia.

Untuk bagian ini, gunakan `df_train` untuk menjawab soal-soal yang diberikan.

In [3]:
splits = {
    "validation": "v1.1/validation-00000-of-00001.parquet",
    "train": "v1.1/train-00000-of-00001.parquet",
    "test": "v1.1/test-00000-of-00001.parquet",
}
df_train = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["validation"])
df_test = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["test"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
df_train.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


### Soal 1.1 (2 poin)

Terdapat beberapa baris yang tidak memiliki passage yang relevan, i.e. semua elemen `is_selected` bernilai nol. Hapus semua baris tersebut.

In [5]:
df_train = df_train[df_train['passages'].apply(lambda x: sum(x['is_selected'])) > 0]
df_val = df_val[df_val['passages'].apply(lambda x: sum(x['is_selected'])) > 0]
assert df_train.shape[0] == 79704
assert df_val.shape[0] == 9706

AttributeError: 'ellipsis' object has no attribute 'shape'

### Soal 1.2 (2 poin)

Lengkapi fungsi untuk menghitung average precision di bawah ini. Nilai parameter `y_true` adalah `is_selected` dari kolom `passages`, sedangkan `y_scores` adalah nilai yang digunakan untuk mengurutkan passage berdasarkan relevansi.

In [ ]:
def average_precision(y_true, y_scores):
    relevant_docs = np.sum(y_true)
    if relevant_docs == 0:
        return 0.0

    sorted_indices = np.argsort(y_scores)[::-1]
    y_true_sorted = y_true[sorted_indices]

    precision_sum = 0.0
    hits = 0
    for i, is_relevant in enumerate(y_true_sorted):
        if is_relevant:
            hits += 1
            precision_sum += hits / (i + 1)

    return precision_sum / relevant_docs


assert np.isclose(average_precision(np.array([1, 0, 1]), np.array([0.9, 0.8, 0.7])), 0.8333, atol=1e-4)
assert np.isclose(average_precision(np.array([1, 0, 1]), np.array([0.95, 0.8, 0.9])), 1.0)
assert np.isclose(average_precision(np.array([1, 0, 0]), np.array([0.0, 0.8, 0.9])), 1 / 3)

### Soal 1.3 (2 poin)

Implementasikan fungsi untuk menghitung cosine similarity.

In [ ]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0.0
    return dot_product / (norm_a * norm_b)


assert np.isclose(cos_sim(np.array([1, 0]), np.array([1, 0])), 1.0)
assert np.isclose(cos_sim(np.array([1, 0]), np.array([0, 1])), 0.0)
assert np.isclose(cos_sim(np.array([1, 2]), np.array([2, 4])), 1.0)

### Soal 1.4 (4 poin)

Gunakan metode TF-IDF untuk mengukur relevansi dari passage berdasarkan *cosine similarity* dari query dan passages. Hitung nilai *mean average precision* dari sistem yang dihasilkan. Dalam sistem ini, Anda tidak perlu membandingkan query dengan semua passages dari setiap baris di tabel. Anda hanya perlu membandingkan query dengan passages yang ada dalam satu baris.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
average_precisions_tfidf = []

for index, row in df_train.iterrows():
    query = row['query']
    passages = row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    if not passages:
        average_precisions_tfidf.append(0.0)
        continue

    # Fit and transform for the current query and its passages
    try:
        all_texts = [query] + passages
        tfidf_matrix = vectorizer.fit_transform(all_texts)
        query_vector = tfidf_matrix[0]
        passage_vectors = tfidf_matrix[1:]

        y_scores = [cos_sim(query_vector.toarray().flatten(), passage_vector.toarray().flatten()) for passage_vector in passage_vectors]
        average_precisions_tfidf.append(average_precision(y_true, np.array(y_scores)))
    except ValueError: # Happens if vocabulary is empty (e.g. all stopwords)
        average_precisions_tfidf.append(0.0)

map_tfidf = np.mean(average_precisions_tfidf)
print(f"MAP with TF-IDF: {map_tfidf}")

### Soal 1.5 (2 poin)

Implementasikan sistem yang serupa seperti soal 1.3, tetapi kali ini gunakan BM25 untuk menghitung skornya. Gunakan implementasi seperti yang dicontohkan [di sini](https://pypi.org/project/rank-bm25/).

In [ ]:
from rank_bm25 import BM25Okapi

average_precisions_bm25 = []

for index, row in df_train.iterrows():
    query = row['query']
    passages = row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    if not passages:
        average_precisions_bm25.append(0.0)
        continue

    tokenized_corpus = [doc.split(" ") for doc in passages]
    tokenized_query = query.split(" ")

    try:
        bm25 = BM25Okapi(tokenized_corpus)
        y_scores = bm25.get_scores(tokenized_query)
        average_precisions_bm25.append(average_precision(y_true, y_scores))
    except ValueError: # Can happen with empty documents or queries after tokenization
        average_precisions_bm25.append(0.0)
    except IndexError: # Can happen if all documents are empty, BM25 can't initialize
        average_precisions_bm25.append(0.0)

map_bm25 = np.mean(average_precisions_bm25)
print(f"MAP with BM25: {map_bm25}")

### Soal 1.6 (3 poin)

Bagaimana nilai MAP yang Anda dapatkan dari BM25 jika dibandingkan dengan tf-idf? Mengapa hasilnya seperti itu?

Nilai MAP yang dihasilkan oleh BM25 umumnya lebih tinggi dibandingkan dengan TF-IDF. Hal ini disebabkan oleh beberapa faktor:
1.  **Penanganan Panjang Dokumen**: BM25 secara eksplisit memperhitungkan panjang dokumen dalam perhitungannya. Dokumen yang lebih pendek yang mengandung term pencarian akan mendapatkan skor yang lebih tinggi dibandingkan dokumen panjang dengan frekuensi term yang sama. TF-IDF tidak secara langsung mengakomodasi ini dengan cara yang sama.
2.  **Saturasi Frekuensi Term**: BM25 memiliki komponen saturasi frekuensi term (parameter k1). Ini berarti bahwa setelah frekuensi term mencapai titik tertentu, peningkatan lebih lanjut dalam frekuensi term tersebut memberikan peningkatan skor yang semakin berkurang. Hal ini membantu mencegah term yang sangat umum mendominasi skor relevansi secara tidak proporsional, yang bisa terjadi pada TF-IDF.
3.  **Parameter yang Dapat Disesuaikan**: BM25 memiliki parameter (k1 dan b) yang dapat disesuaikan (tuned) untuk dataset tertentu, yang memungkinkannya beradaptasi lebih baik dengan karakteristik data tersebut. TF-IDF memiliki lebih sedikit fleksibilitas dalam hal ini.

Secara umum, BM25 dianggap sebagai penyempurnaan dari model probabilitas dan seringkali memberikan hasil yang lebih baik dalam tugas-tugas pencarian informasi dibandingkan TF-IDF standar, terutama ketika panjang dokumen bervariasi dan ada kebutuhan untuk mengontrol dampak frekuensi term yang sangat tinggi.

## 2. Dense Matrix Retrieval (10 poin)

### Soal 2.1 (2 poin)

Ambil 500 sampel (baris) dari `df_val`. Hitung nilai MAP dari sistem dengan tf-idf.

In [ ]:
np.random.seed(42)

df_sample = df_val.sample(n=500, random_state=42)

average_precisions_tfidf_sample = []
vectorizer_sample = TfidfVectorizer()

for index, row in df_sample.iterrows():
    query = row['query']
    passages = row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    if not passages:
        average_precisions_tfidf_sample.append(0.0)
        continue
    try:
        all_texts = [query] + passages
        tfidf_matrix = vectorizer_sample.fit_transform(all_texts)
        query_vector = tfidf_matrix[0]
        passage_vectors = tfidf_matrix[1:]

        y_scores = [cos_sim(query_vector.toarray().flatten(), passage_vector.toarray().flatten()) for passage_vector in passage_vectors]
        average_precisions_tfidf_sample.append(average_precision(y_true, np.array(y_scores)))
    except ValueError:
        average_precisions_tfidf_sample.append(0.0)

map_tfidf_sample = np.mean(average_precisions_tfidf_sample)
print(f"MAP with TF-IDF on df_sample: {map_tfidf_sample}")

### Soal 2.2 (3 poin)

Gunakan model `all-MiniLM-L6-v2` untuk menghasilkan embeddings dari query dan passages. Lalu, hitung nilai MAP dari sampel yang dihasilkan di soal 2.1.

In [ ]:
from sentence_transformers import SentenceTransformer

model_minilm = SentenceTransformer('all-MiniLM-L6-v2')
average_precisions_minilm_sample = []

for index, row in df_sample.iterrows():
    query = row['query']
    passages = row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    if not passages:
        average_precisions_minilm_sample.append(0.0)
        continue

    query_embedding = model_minilm.encode(query)
    passage_embeddings = model_minilm.encode(passages)

    y_scores = [cos_sim(query_embedding, passage_embedding) for passage_embedding in passage_embeddings]
    average_precisions_minilm_sample.append(average_precision(y_true, np.array(y_scores)))

map_minilm_sample = np.mean(average_precisions_minilm_sample)
print(f"MAP with all-MiniLM-L6-v2 on df_sample: {map_minilm_sample}")

### Soal 2.3 (3 poin)

Bagaimana nilai MAP yang Anda dapatkan dari sistem dengan embeddings jika dibandingkan dengan tf-idf? Tuliskan 3 kelebihan dari sistem IR dengan embeddings jika dibandingkan dengan tf-idf.

Nilai MAP yang didapatkan dari sistem dengan embeddings (menggunakan `all-MiniLM-L6-v2`) kemungkinan besar akan lebih tinggi dibandingkan dengan TF-IDF. Ini karena model embedding mampu menangkap makna semantik dan hubungan kontekstual antar kata, tidak hanya berdasarkan frekuensi kata.

Berikut adalah 3 kelebihan sistem IR dengan embeddings dibandingkan dengan TF-IDF:
1.  **Pemahaman Semantik**: Embeddings dapat memahami sinonim dan parafrase. Misalnya, query "mobil murah" mungkin akan menemukan dokumen yang berisi "kendaraan ekonomis" karena model embedding memahami bahwa kedua frasa tersebut memiliki makna yang serupa. TF-IDF akan kesulitan menangkap hubungan ini karena hanya melihat kesamaan kata secara literal.
2.  **Penanganan Kata di Luar Kosakata (Out-of-Vocabulary/OOV)**: Model embedding modern, terutama yang berbasis subword (seperti BPE atau WordPiece), dapat menangani kata-kata yang tidak ada dalam vocabulary pelatihan dengan lebih baik. Mereka dapat merepresentasikan kata-kata baru berdasarkan subword yang sudah dikenal. TF-IDF akan mengabaikan kata-kata OOV atau memperlakukannya sebagai token unik tanpa informasi semantik.
3.  **Representasi Kontekstual**: Embeddings, terutama yang dihasilkan oleh model Transformer, dapat menghasilkan representasi kata atau kalimat yang berbeda tergantung pada konteksnya. Ini berarti kata yang sama bisa memiliki embedding yang berbeda jika digunakan dalam makna yang berbeda. TF-IDF tidak memiliki kemampuan ini; setiap kata memiliki representasi yang tetap terlepas dari konteks penggunaannya.

### Soal 2.4 (2 poin)

Karena dataset yang digunakan sudah menangani bagian *retrieval*, gunakan model `CrossEncoder` untuk memberikan skor dari passages yang ada. Hitung nilai MAP-nya. Apakah Anda mendapatkan nilai MAP yang lebih baik?

In [ ]:
from sentence_transformers import CrossEncoder

cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
average_precisions_cross_encoder_sample = []

for index, row in df_sample.iterrows():
    query = row['query']
    passages = row['passages']['passage_text']
    y_true = np.array(row['passages']['is_selected'])

    if not passages:
        average_precisions_cross_encoder_sample.append(0.0)
        continue

    # CrossEncoder expects a list of [query, passage] pairs
    sentence_pairs = [[query, passage] for passage in passages]
    y_scores = cross_encoder_model.predict(sentence_pairs)

    average_precisions_cross_encoder_sample.append(average_precision(y_true, y_scores))

map_cross_encoder_sample = np.mean(average_precisions_cross_encoder_sample)
print(f"MAP with CrossEncoder on df_sample: {map_cross_encoder_sample}")

print(f"\nComparison of MAP scores on df_sample:")
print(f"TF-IDF: {map_tfidf_sample}")
print(f"all-MiniLM-L6-v2 (Bi-Encoder): {map_minilm_sample}")
print(f"Cross-Encoder: {map_cross_encoder_sample}")

print("\nNilai MAP yang didapatkan dengan CrossEncoder kemungkinan besar akan menjadi yang tertinggi. CrossEncoder memproses query dan passage secara bersamaan, memungkinkan interaksi yang lebih dalam antar token dan pemahaman konteks yang lebih baik dibandingkan Bi-Encoder (seperti all-MiniLM-L6-v2) yang menghasilkan embedding secara terpisah. Meskipun lebih lambat karena harus memproses setiap pasangan query-passage, akurasinya cenderung lebih superior untuk tugas re-ranking.")

## 3. Retrieval-Augmented Generation (25 poin)

*Acknowledgement:* Data yang digunakan pada bagian ini dikumpulkan oleh [Miguel Escobar Varela](https://miguelescobar.com/) dan tersedia secara terbuka di [HuggingFace](https://huggingface.co/datasets/mevsg/warisan-classification-data-v1).

In [ ]:
splits = {
    "train": "data/train-00000-of-00001.parquet",
    "test": "data/test-00000-of-00001.parquet",
}
df = pd.concat(
    [
        pd.read_parquet(
            "hf://datasets/mevsg/warisan-classification-data-v1/" + splits["train"]
        ),
        pd.read_parquet(
            "hf://datasets/mevsg/warisan-classification-data-v1/" + splits["test"]
        ),
    ],
    ignore_index=True
)

### Soal 3.1 (3 poin)

Dengan menggunakan model `Llama-3.2-1B`, berikan pertanyaan berikut sebagai prompt dan tampilkan jawabannya. Apa yang dapat Anda amati dari hasilnya?

In [ ]:
query = "Apa saja warisan budaya tak benda dari Jawa Barat?"

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "meta-llama/Llama-3.2-1B"
tokenizer_llama = AutoTokenizer.from_pretrained(model_name)
model_llama = AutoModelForCausalLM.from_pretrained(model_name)

inputs = tokenizer_llama(query, return_tensors="pt")

# Generate text
with torch.no_grad():
    outputs = model_llama.generate(**inputs, max_new_tokens=200, pad_token_id=tokenizer_llama.eos_token_id) # Added pad_token_id

response = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
print(f"Query: {query}")
print(f"Llama-3.2-1B Response (without RAG):\n{response}")

Jawaban dari model Llama-3.2-1B tanpa RAG kemungkinan akan bersifat umum atau bahkan menghasilkan informasi yang tidak akurat (halusinasi). Model ini, meskipun besar, mungkin tidak memiliki pengetahuan spesifik dan terkini mengenai daftar warisan budaya tak benda dari Jawa Barat yang ada dalam dataset `warisan-classification-data-v1`.

Pengamatan yang mungkin muncul:
1.  **Jawaban Generik**: Model mungkin memberikan contoh-contoh umum warisan budaya Indonesia atau Jawa secara umum, tanpa menyebutkan secara spesifik yang terdaftar resmi atau yang ada di dataset.
2.  **Ketidakakuratan/Halusinasi**: Model bisa saja menyebutkan beberapa item yang terdengar plausibel namun sebenarnya bukan warisan budaya tak benda dari Jawa Barat, atau bahkan mengarang nama-nama warisan.
3.  **Kurangnya Detail**: Jawaban mungkin tidak mendalam dan tidak memberikan konteks atau penjelasan lebih lanjut mengenai warisan yang disebutkan.
4.  **Ketergantungan pada Data Pelatihan**: Jawaban sangat bergantung pada data yang digunakan untuk melatih model Llama tersebut. Jika data pelatihan tidak mencakup informasi detail tentang warisan budaya Jawa Barat, maka model tidak akan bisa memberikannya.

Hal ini menunjukkan keterbatasan LLM standar ketika dihadapkan pada pertanyaan yang membutuhkan pengetahuan domain spesifik yang mungkin tidak tercakup secara komprehensif dalam data pelatihannya. Ini menjadi dasar mengapa RAG diperlukan, yaitu untuk menyediakan konteks yang relevan dan faktual kepada LLM.

### Soal 3.2 (2 poin)

Sekarang, Anda akan membangun sistem retrieval-augmented generation (RAG) sederhana. Bagian pertama dari model ini adalah membuat embeddings dari corpus yang kita gunakan.

Bentuk corpus dari kolom `texts`. Urai teks menggunakan delimiter `\n\n`, simpan hanya teks dengan ukuran lebih dari 100 karakter.

In [ ]:
corpus = []
for text_entry in df['texts']:
    paragraphs = text_entry.split('\n\n')
    for paragraph in paragraphs:
        if len(paragraph) > 100:
            corpus.append(paragraph.strip())

print(f"Jumlah dokumen dalam korpus: {len(corpus)}")
assert len(corpus) == 12037

### Soal 3.3.a (2 poin)

Buat embeddings dari corpus dengan model `all-MiniLM-L6-v2`.

In [ ]:
model_minilm_rag = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings_minilm = model_minilm_rag.encode(corpus, convert_to_tensor=True, show_progress_bar=True)
print(f"Shape of MiniLM corpus embeddings: {corpus_embeddings_minilm.shape}")

### Soal 3.3.b (2 poin)

Buat embeddings dari corpus dengan model `intfloat/multilingual-e5-small`.

In [ ]:
model_e5_rag = SentenceTransformer('intfloat/multilingual-e5-small')
corpus_embeddings_e5 = model_e5_rag.encode(corpus, convert_to_tensor=True, show_progress_bar=True)
print(f"Shape of E5 corpus embeddings: {corpus_embeddings_e5.shape}")

### Soal 3.3.c (4 poin)

Buat embedding dari variabel `query` dengan menggunakan dua model dari soal 3.3.a dan 3.3.b, lalu ambil lima dokumen dengan skor paling tinggi dari `corpus`. Anda dapat memanfaatkan fungsi *semantic search* dari Sentence Transformers.

In [ ]:
from sentence_transformers import util

query_rag = "Apa saja warisan budaya tak benda dari Jawa Barat?"
top_k = 5

# Semantic Search with all-MiniLM-L6-v2
query_embedding_minilm = model_minilm_rag.encode(query_rag, convert_to_tensor=True)
hits_minilm = util.semantic_search(query_embedding_minilm, corpus_embeddings_minilm, top_k=top_k)
hits_minilm = hits_minilm[0]  # Get the hits for the first query

print(f"Query: {query_rag}")
print(f"\nTop {top_k} results from all-MiniLM-L6-v2:")
for hit in hits_minilm:
    print(f"  Score: {hit['score']:.4f} - {corpus[hit['corpus_id']]}\n")

# Semantic Search with intfloat/multilingual-e5-small
query_embedding_e5 = model_e5_rag.encode(query_rag, convert_to_tensor=True)
hits_e5 = util.semantic_search(query_embedding_e5, corpus_embeddings_e5, top_k=top_k)
hits_e5 = hits_e5[0]  # Get the hits for the first query

print(f"\nTop {top_k} results from intfloat/multilingual-e5-small:")
for hit in hits_e5:
    print(f"  Score: {hit['score']:.4f} - {corpus[hit['corpus_id']]}\n")

### Soal 3.3.d (2 poin)

Apa yang dapat Anda amati dari hasil kedua model di atas? Apa yang menyebabkan perbedaan tersebut?

Perbedaan hasil antara `all-MiniLM-L6-v2` dan `intfloat/multilingual-e5-small` dapat disebabkan oleh beberapa faktor utama:

1.  **Data Pelatihan dan Kemampuan Multibahasa**:
    *   `all-MiniLM-L6-v2` adalah model yang sangat baik untuk bahasa Inggris dan dilatih pada dataset berbahasa Inggris yang besar. Meskipun dapat menangani bahasa lain sampai batas tertentu melalui kemiripan token atau transfer learning implisit, performanya mungkin tidak seoptimal pada bahasa non-Inggris.
    *   `intfloat/multilingual-e5-small` secara eksplisit dirancang dan dilatih untuk menjadi model multibahasa. Model ini dilatih pada data dari banyak bahasa, termasuk bahasa Indonesia. Oleh karena itu, untuk query dan korpus dalam bahasa Indonesia (seperti dalam kasus ini), `multilingual-e5-small` cenderung memiliki pemahaman semantik yang lebih baik dan representasi yang lebih akurat.

2.  **Arsitektur Model dan Ukuran**:
    *   Meskipun keduanya adalah model Transformer, perbedaan dalam arsitektur spesifik, jumlah parameter (Meskipun `multilingual-e5-small` adalah versi 'small', arsitektur E5 mungkin berbeda dari MiniLM), dan detail pelatihan lainnya dapat memengaruhi kualitas embedding yang dihasilkan untuk tugas tertentu.

3.  **Spesialisasi Tugas Pelatihan**:
    *   Model E5 (Embedding for Everything) dilatih dengan tujuan untuk menghasilkan embedding yang baik untuk berbagai tugas, termasuk pencarian semantik (retrieval). Cara pelatihannya mungkin lebih dioptimalkan untuk membedakan antara query dan passage yang relevan dalam konteks multibahasa.

**Observasi yang Diharapkan**:
*   **Relevansi Hasil**: Kemungkinan besar `intfloat/multilingual-e5-small` akan memberikan hasil pencarian yang lebih relevan untuk query berbahasa Indonesia "Apa saja warisan budaya tak benda dari Jawa Barat?". Dokumen yang diambil mungkin lebih akurat mencerminkan konten yang berkaitan dengan Jawa Barat dan warisan budayanya.
*   **Skor Kepercayaan (Score)**: Skor yang diberikan oleh `multilingual-e5-small` mungkin lebih terkalibrasi dengan baik untuk relevansi dalam konteks bahasa Indonesia.
*   **Nuansa Bahasa**: `multilingual-e5-small` mungkin lebih mampu menangkap nuansa bahasa Indonesia dan istilah-istilah spesifik yang terkait dengan budaya lokal dibandingkan `all-MiniLM-L6-v2` yang fokus utama pelatihannya adalah bahasa Inggris.

Secara singkat, karena query dan korpusnya dalam bahasa Indonesia, model yang secara eksplisit dilatih untuk multibahasa seperti `intfloat/multilingual-e5-small` diharapkan memberikan performa yang lebih baik dalam hal relevansi pencarian semantik.

### Soal 3.4 (3 poin)

Dengan menggunakan model `intfloat/multilingual-e5-small` untuk mencari dokumen yang relevan, rangkum hasilnya dengan model `Llama-3.2-1B`.

Catatan: Anda mungkin butuh rekayasa *prompt* agar mendapatkan hasil yang diharapkan, e.g. membuang informasi yang tidak relevan, rangkuman tertulis dalam format yang rapi.

In [ ]:
retrieved_passages_e5 = [corpus[hit['corpus_id']] for hit in hits_e5]
context_for_llama = "\n\n".join(retrieved_passages_e5)

prompt_template = f"""
Berdasarkan informasi berikut:
{context}

Jawab pertanyaan ini: {question}
Berikan jawaban yang ringkas dan hanya berdasarkan informasi yang diberikan. Sebutkan beberapa warisan budaya tak benda dari Jawa Barat.
"""

prompt_rag = prompt_template.format(context=context_for_llama, question=query_rag)

inputs_rag = tokenizer_llama(prompt_rag, return_tensors="pt")

# Generate text with RAG
with torch.no_grad():
    outputs_rag = model_llama.generate(**inputs_rag, max_new_tokens=250, pad_token_id=tokenizer_llama.eos_token_id, temperature=0.7, do_sample=True) # Added temperature and do_sample for better generation

response_rag = tokenizer_llama.decode(outputs_rag[0], skip_special_tokens=True)

print(f"Query: {query_rag}")
print(f"\nLlama-3.2-1B Response (with RAG using multilingual-e5-small):\n{response_rag}")

### Soal 3.5 (2 poin)

Jelaskan dua contoh metrik evaluasi yang dapat digunakan untuk mengukur kualitas bagian generation dari sebuah sistem RAG.

Berikut adalah dua contoh metrik evaluasi yang dapat digunakan untuk mengukur kualitas bagian *generation* dari sebuah sistem RAG, beserta penjelasannya:

1.  **BLEU (Bilingual Evaluation Understudy)**:
    *   **Deskripsi**: BLEU adalah metrik yang awalnya dirancang untuk mengevaluasi kualitas terjemahan mesin, tetapi juga sering digunakan untuk mengevaluasi teks yang dihasilkan oleh model generatif lainnya, termasuk dalam RAG. BLEU bekerja dengan membandingkan n-gram (urutan kata yang berdekatan) dari teks yang dihasilkan model dengan n-gram dari satu atau lebih referensi teks (jawaban ideal yang ditulis manusia). Skor BLEU berkisar antara 0 dan 1 (atau 0 hingga 100), di mana skor yang lebih tinggi menunjukkan kesamaan yang lebih besar dengan referensi.
    *   **Kelebihan**: Cepat dihitung, banyak digunakan sehingga mudah untuk membandingkan dengan penelitian lain, dan berkorelasi cukup baik dengan penilaian manusia untuk beberapa tugas.
    *   **Keterbatasan**: Sangat bergantung pada kualitas referensi. Kurang baik dalam menangkap makna semantik atau parafrase (jika teks yang dihasilkan menggunakan kata-kata berbeda tetapi maknanya sama dengan referensi, skor BLEU bisa rendah). Juga, tidak memperhitungkan kelancaran (fluency) atau koherensi secara langsung.
    *   **Penggunaan dalam RAG**: Digunakan untuk menilai seberapa dekat jawaban yang dihasilkan oleh LLM (setelah diberi konteks dari retriever) dengan jawaban referensi yang dianggap benar dan relevan dengan query serta konteks yang diberikan.

2.  **ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**:
    *   **Deskripsi**: ROUGE adalah serangkaian metrik yang digunakan untuk mengevaluasi ringkasan otomatis dan juga relevan untuk teks yang dihasilkan secara generatif. Berbeda dengan BLEU yang fokus pada presisi n-gram, ROUGE lebih fokus pada recall n-gram. Varian yang umum digunakan meliputi:
        *   `ROUGE-N`: Mengukur tumpang tindih n-gram (misalnya, `ROUGE-1` untuk unigram, `ROUGE-2` untuk bigram).
        *   `ROUGE-L`: Mengukur Longest Common Subsequence (LCS) antara teks yang dihasilkan dan referensi. Ini menangkap kesamaan struktur kalimat.
    *   **Kelebihan**: Baik dalam menilai seberapa banyak informasi penting dari referensi yang muncul dalam teks yang dihasilkan. `ROUGE-L` dapat menangkap kesamaan urutan kata meskipun tidak persis sama.
    *   **Keterbatasan**: Seperti BLEU, sangat bergantung pada referensi. Mungkin tidak sepenuhnya menangkap kualitas semantik jika kata-kata yang digunakan berbeda. Kurang menilai aspek seperti faktualitas (jika tidak ada dalam referensi) atau kelancaran.
    *   **Penggunaan dalam RAG**: Digunakan untuk menilai apakah jawaban yang dihasilkan oleh LLM mencakup poin-poin penting yang diharapkan ada dalam jawaban berdasarkan query dan konteks yang diambil. Ini membantu mengukur sejauh mana LLM berhasil memanfaatkan informasi dari dokumen yang diambil untuk menghasilkan jawaban yang komprehensif.

Metrik lain yang juga bisa dipertimbangkan termasuk **METEOR**, **CIDEr** (untuk deskripsi gambar, tapi konsepnya bisa diadaptasi), atau metrik berbasis embedding seperti **BERTScore** yang mencoba menangkap kesamaan semantik lebih baik daripada metrik berbasis n-gram tradisional. Evaluasi manusia juga tetap menjadi standar emas untuk menilai kualitas generasi teks secara menyeluruh.

### Soal 3.6 (5 poin)

Lakukan eksplorasi tambahan untuk bagian ini, misalnya:
1. Evaluasi sistem Anda dengan query yang berbeda
2. Gunakan model berbeda untuk bagian IR dari sistem
3. Gunakan model berbeda sebagai LLM
4. Gunakan OpenAI API tanpa komponen IR untuk mengevaluasi query

In [ ]:
# Eksplorasi Tambahan (Contoh)

# 1. Evaluasi sistem dengan query yang berbeda
new_query = "Sebutkan beberapa tarian tradisional dari Sumatera Utara."
print(f"--- Mengevaluasi dengan Query Baru: '{new_query}' ---")

#   a. Dapatkan konteks dengan model retrieval (multilingual-e5-small)
new_query_embedding_e5 = model_e5_rag.encode(new_query, convert_to_tensor=True)
new_hits_e5 = util.semantic_search(new_query_embedding_e5, corpus_embeddings_e5, top_k=top_k)
new_hits_e5 = new_hits_e5[0]
retrieved_passages_new_query = [corpus[hit['corpus_id']] for hit in new_hits_e5]
context_for_llama_new_query = "\n\n".join(retrieved_passages_new_query)

print(f"Dokumen yang diambil untuk query baru:")
for i, passage in enumerate(retrieved_passages_new_query):
    print(f"  {i+1}. {passage[:150]}...") # Print first 150 chars

#   b. Hasilkan jawaban dengan Llama-3.2-1B menggunakan konteks baru
prompt_rag_new_query = prompt_template.format(context=context_for_llama_new_query, question=new_query)
inputs_rag_new_query = tokenizer_llama(prompt_rag_new_query, return_tensors="pt")

with torch.no_grad():
    outputs_rag_new_query = model_llama.generate(**inputs_rag_new_query, max_new_tokens=250, pad_token_id=tokenizer_llama.eos_token_id, temperature=0.7, do_sample=True)

response_rag_new_query = tokenizer_llama.decode(outputs_rag_new_query[0], skip_special_tokens=True)
print(f"\nLlama-3.2-1B Response untuk query baru (with RAG):")
print(response_rag_new_query)

# Komentar Tambahan:
# Untuk eksplorasi lebih lanjut, bisa juga:
# 2. Gunakan model berbeda untuk IR: Misalnya, coba BM25 atau TF-IDF lagi pada korpus 'warisan' dan bandingkan dokumen yang diambilnya.
# 3. Gunakan model LLM berbeda: Jika akses tersedia, coba model seperti Llama-3-8B atau model lain yang lebih besar/kecil untuk melihat perbedaan kualitas generasi.
# 4. Gunakan OpenAI API tanpa RAG: Kirim 'new_query' ke API OpenAI (misal GPT-3.5 atau GPT-4) tanpa memberikan konteks dari korpus kita, lalu bandingkan jawabannya dengan hasil RAG.
#    Ini akan menunjukkan seberapa baik pengetahuan umum model tersebut dibandingkan dengan sistem RAG yang memiliki akses ke data spesifik.
#    Contoh (membutuhkan setup API Key OpenAI):
#    # from openai import OpenAI
#    # client = OpenAI(api_key='YOUR_OPENAI_API_KEY')
#    # response_openai = client.chat.completions.create(
#    #    model="gpt-3.5-turbo",
#    #    messages=[
#    #        {"role": "system", "content": "You are a helpful assistant."},
#    #        {"role": "user", "content": new_query}
#    #    ]
#    # )
#    # print(f"\nOpenAI GPT-3.5 Response (without RAG):\n{response_openai.choices[0].message.content}")
print("\nEksplorasi di atas menunjukkan bagaimana sistem RAG merespons query yang berbeda dengan mengambil konteks yang relevan dari korpus yang sama dan menggunakan LLM untuk menghasilkan jawaban berdasarkan konteks tersebut.")